In [10]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [11]:
ENCODING_DICT = {
            'a': 0,
            'f': 1,
            'e': 2,
            'c': 3,
            'b': 4,
            'h': 5,
            'v': 6,
            'z': 7,
            '2': 8,
            'x': 9,
            'g': 10,
            'm': 11,
            'r': 12,
            'u': 13,
            'p': 14,
            's': 15,
            'd': 16,
            'n': 17,
            '6': 18,
            'k': 19,
            't': 20
            }

DECODING_DICT = {
            0: 'a',
            1: 'f',
            2: 'e',
            3: 'c',
            4: 'b',
            5: 'h',
            6: 'v',
            7: 'z',
            8: '2',
            9: 'x',
            10: 'g',
            11: 'm',
            12: 'r',
            13: 'u',
            14: 'p',
            15: 's',
            16: 'd',
            17: 'n',
            18: '6',
            19: 'k',
            20: 't'
            }

In [12]:
def imagepath_to_captcha(imagepath):
    return imagepath.split("_")[1].split(".")[0]

def to_onehot(captcha):
    label = np.zeros((len(captcha), len(ENCODING_DICT)))
    for index, char in enumerate(captcha):
        label[index, ENCODING_DICT[char]] = 1
    return tf.convert_to_tensor(label)

def captcha_from_onehot(tensor):
    label = []
    for index in tf.argmax(tensor, 1).numpy():
        label.append(DECODING_DICT[index])
    return "".join(label)

def load_dataset(filepath):
    filenames = os.listdir(filepath)
    x, y = [], []
    for file in filenames:
        imagepath = filepath + "/" + file
        image = tf.io.read_file(imagepath)
        image = tf.image.decode_image(image)
        pixels = tf.image.convert_image_dtype(image, tf.float32)
        # To show images: 
        # plt.imshow(tf.reshape(pixels, image.shape))
        # plt.title('Image as Pixels')
        # plt.show()
        x.append(pixels)
        captcha = imagepath_to_captcha(file).lower()
        y.append(to_onehot(captcha))
    return (tf.convert_to_tensor(x), tf.convert_to_tensor(y))


# Read input data

In [13]:
train_x, train_y = load_dataset("dataset/train")
test_x, test_y = load_dataset("dataset/test")

print("Training input shape:", train_x.shape)
print("Training output shape:", train_y.shape)
print("Testing input shape:", test_x.shape)
print("Testing output shape:", test_y.shape)

Training input shape: (8501, 50, 250, 3)
Training output shape: (8501, 6, 21)
Testing input shape: (1500, 50, 250, 3)
Testing output shape: (1500, 6, 21)


In [29]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=(50, 250, 3)))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(1008, activation='relu'))
model.add(keras.layers.Dense(504, activation='relu'))
# model.add(keras.layers.Dense(504, activation='relu'))
model.add(keras.layers.Dense(252, activation='relu'))
model.add(keras.layers.Dense(126, activation='softmax'))
model.add(keras.layers.Reshape((6, 21)))


model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 49, 249, 32)       416       
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 24, 124, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 23, 123, 32)       4128      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 11, 61, 32)        0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 21472)             0         
                                                                 
 dense_16 (Dense)            (None, 504)              

In [30]:
model.compile(
    optimizer = "adam",
    loss = 'categorical_crossentropy'
)

In [31]:
history = model.fit(train_x, train_y,
                    epochs = 4)

print(history.history)

Epoch 1/4
266/266 [==============================] - 90s 334ms/step - loss: 2.8090
Epoch 2/4
266/266 [==============================] - 97s 365ms/step - loss: 1.5196
Epoch 3/4
266/266 [==============================] - 102s 383ms/step - loss: 0.6889
Epoch 4/4
266/266 [==============================] - 107s 402ms/step - loss: 0.2977
{'loss': [2.8089828491210938, 1.5195749998092651, 0.6889254450798035, 0.29772934317588806]}


In [32]:
model.evaluate(test_x, test_y)

47/47 [==============================] - 4s 62ms/step - loss: 1.1206


1.1205676794052124

In [33]:
y_pred = model.predict(test_x)
met, notmet = 0, 0
for i in range(len(y_pred)):
    if captcha_from_onehot(y_pred[i]) != captcha_from_onehot(test_y[i]):
        print(captcha_from_onehot(y_pred[i]), captcha_from_onehot(test_y[i]))
        notmet += 1
    else:
        met += 1

print(met, notmet)

47/47 [==============================] - 3s 60ms/step
ztpsk6 ztasxg
akebpv xkespv
uzrbxt uzrsav
pbetnu rfefnu
ez2tdt 2z2tdt
reurc6 rv6kvs
henpzx xeufz2
gpbkz2 gfbkz2
ezpxnk etpkmk
vbxfrn ubkfpm
ckhhk6 gxhhku
xbuftu xdubtu
rtks2f rtas2f
eutvdk 6utvdr
2tvakk 2tmakk
zbzxmp zd2amf
6vednk 6vedmk
guhfk6 6grfkg
arsv2v axsv2v
usucsx usgssh
bhdbcu hhdbcg
akr2zb xkrczb
mmzcfr mmzdfk
ptrxst ttpcht
tevkxc eeanxd
k2bkns x2hkms
kgbcvr cgbcvr
cfuc6s cfuu6s
pvsmze pvsnze
e2pvfu e2pnfu
2kngnu 2knsnu
baczu2 exczu2
ebbc2m bseczv
zbvctf zbmctf
c2ptee c2pgre
puevfx fuevfx
a2bfss x2bfs6
2fhuca 2fhuua
a6ztpk mgzepg
puedxc eubdkd
dfxcnh dfxxnh
ahvdzf xhvdzf
zvdbxr zvdbkr
kfk6f6 kfk6fg
6rd2tt 6rdctx
g6excv grzxzv
stpmcz stfuez
dsrcnt dsscnt
acd2va uddtvh
abnkfr ahkkxt
agn2ds rgv2ds
axnctr rxngfr
6gbk6d 6gbked
pzetru fzehbu
p6v266 pev26e
pszd6c fszdgg
2zte2z 2zee2z
varcsn varusn
fgupsn fgupsm
p6ffgx p6fp6v
bpzkzm bpzf2m
gceetb gcsbtr
egtzec bufz6k
dkvepc dkvspc
p6dtne f6danz
kcr2va kgr2va
aaptsv rapvsv
heages h